In [1]:
!pip install pytorch-lightning
!pip install tensorboard
#!pip install numpy<2.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.5/849.5 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 57.7 MB/s eta 0:00:00


In [2]:
import os
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GroupShuffleSplit

from pytorch_lightning.loggers import TensorBoardLogger

pl.seed_everything(42)

INFO:lightning_fabric.utilities.seed:Seed set to 42


42

Pobieranie datasetu CMAPSS

In [3]:
! git clone https://github.com/edwardzjl/CMAPSSData.git

Cloning into 'CMAPSSData'...
remote: Enumerating objects: 16, done.
remote: Total 16 (delta 0), reused 0 (delta 0), pack-reused 16 (from 1)
Receiving objects: 100% (16/16), 11.96 MiB | 13.49 MiB/s, done.
Resolving deltas: 100% (1/1), done.


Implementacja potrzebnych klas

In [4]:
class CMAPSS_Preprocessor:
    def __init__(self, data_path, sequence_length, alpha=0.25, max_rul=125):
        self.data_path = data_path
        self.sequence_length = sequence_length
        self.alpha = alpha  # Współczynnik wygładzania
        self.max_rul = max_rul # Przycinanie RUL

        self.index_cols = ['unit_nr', 'time_cycles']
        self.setting_cols = ['os_1', 'os_2', 'os_3']

        # Definicja wszystkich sensorów oraz tych wybranych w artykule
        self.all_sensor_cols = ['s' + str(i) for i in range(1, 22)]

        # Zgodnie z artykułem dla FD001 wybrano tylko te sensory (CSC > 0.75):
        # S2, S3, S4, S7, S8, S11, S12, S13, S15, S17, S20, S21
        self.selected_sensors = ['s2', 's3', 's4', 's7', 's8', 's11',
                                 's12', 's13', 's15', 's17', 's20', 's21']

        self.cols = self.index_cols + self.setting_cols + self.all_sensor_cols

        self.scaler = MinMaxScaler(feature_range=(0, 1))

    def process(self, file_name='FD001'):

        train_df = pd.read_csv(f'{self.data_path}/train_{file_name}.txt', sep=r'\s+', header=None, names=self.cols)
        test_df = pd.read_csv(f'{self.data_path}/test_{file_name}.txt', sep=r'\s+', header=None, names=self.cols)
        test_rul_df = pd.read_csv(f'{self.data_path}/RUL_{file_name}.txt', sep=r'\s+', header=None, names=['RUL'])

        # Obliczenie etykiet RUL dla treningu
        train_df = self._add_rul(train_df, is_test=False)

        # Wygładzanie danych (Exponential Smoothing)
        train_df = self._smooth_data(train_df)
        test_df = self._smooth_data(test_df)

        # Bierzemy wybrane sensory
        feats = self.selected_sensors

        # Normalizacja
        self.scaler.fit(train_df[feats])
        train_df[feats] = self.scaler.transform(train_df[feats])
        test_df[feats] = self.scaler.transform(test_df[feats])


        # Podział zbioru treningowego na Train (90) i Val (10)
        splitter = GroupShuffleSplit(n_splits=1, train_size=0.9, random_state=42)
        train_idx, val_idx = next(splitter.split(train_df, groups=train_df['unit_nr']))

        real_train_df = train_df.iloc[train_idx].copy()
        real_val_df = train_df.iloc[val_idx].copy()


        # Generowanie okien czasowych (Sliding Window)
        # Generujemy osobno dla Train i Val
        X_train, y_train = self._gen_sequence(real_train_df, feats)
        X_val, y_val = self._gen_sequence(real_val_df, feats)

        # Dla testu w C-MAPSS bierzemy tylko OSTATNIE okno,
        # bo plik RUL_FD001.txt zawiera tylko jedną liczbę dla każdego silnika (RUL na samym końcu).
        X_test, y_test = self._gen_test_sequence(test_df, test_rul_df, feats)

        return X_train, y_train, X_val, y_val, X_test, y_test

    def _add_rul(self, df, is_test=False):
        max_life = df.groupby('unit_nr')['time_cycles'].transform('max')
        df['RUL'] = max_life - df['time_cycles']

        # Implementacja Piecewise Linear RUL (ucinamy powyżej 125)
        df['RUL'] = df['RUL'].clip(upper=self.max_rul)
        return df

    def _smooth_data(self, df):
        # Grupowanie po 'unit_nr'
        # Wygładzanie nie może przenieść się z końca Silnika 1 na początek Silnika 2.
        df[self.all_sensor_cols] = df.groupby('unit_nr')[self.all_sensor_cols].transform(
            lambda x: x.ewm(alpha=self.alpha, adjust=False).mean()
        )
        return df

    def _gen_sequence(self, df, feature_cols):
        X, y = [], []
        data_array = df[feature_cols].values
        target_array = df['RUL'].values
        unit_ids = df['unit_nr'].values

        for i in range(len(df) - self.sequence_length):
            if unit_ids[i] == unit_ids[i + self.sequence_length]:
                X.append(data_array[i : i + self.sequence_length])
                y.append(target_array[i + self.sequence_length - 1])

        return np.array(X), np.array(y)

    def _gen_test_sequence(self, test_df, truth_df, feature_cols):
      # Dla zbioru testowego bierzemy tylko OSTATNIE cykle każdego silnika
        # I przypisujemy mu prawdziwy RUL z pliku z etykietami
        X, y = [], []

        true_ruls = truth_df['RUL'].values

        for unit_id in test_df['unit_nr'].unique():
            temp_df = test_df[test_df['unit_nr'] == unit_id]

            if len(temp_df) >= self.sequence_length:
                window = temp_df[feature_cols].values[-self.sequence_length:]
                X.append(window)
                y.append(true_ruls[unit_id - 1])

        return np.array(X), np.array(y)

In [5]:
class CMAPSSDataModule(pl.LightningDataModule):
    def __init__(self, data_path, batch_size, sequence_length):
        super().__init__()
        self.data_path = data_path
        self.batch_size = batch_size
        self.sequence_length = sequence_length
        self.preprocessor = CMAPSS_Preprocessor(data_path, sequence_length)

    def setup(self, stage=None):
        X_train_np, y_train_np, X_val_np, y_val_np, X_test_np, y_test_np = self.preprocessor.process('FD001')

        self.train_X = torch.tensor(X_train_np, dtype=torch.float32)
        self.train_y = torch.tensor(y_train_np, dtype=torch.float32).unsqueeze(1)

        self.val_X = torch.tensor(X_val_np, dtype=torch.float32)
        self.val_y = torch.tensor(y_val_np, dtype=torch.float32).unsqueeze(1)

        self.test_X = torch.tensor(X_test_np, dtype=torch.float32)
        self.test_y = torch.tensor(y_test_np, dtype=torch.float32).unsqueeze(1)

        print(f"Dane przygotowane!")
        print(f"Trening (90 silników): {self.train_X.shape}")
        print(f"Walidacja (10 silników): {self.val_X.shape}")
        print(f"Test (100 silników, ostatnie okna): {self.test_X.shape}")

    def train_dataloader(self):
        dataset = torch.utils.data.TensorDataset(self.train_X, self.train_y)
        return DataLoader(dataset, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        dataset = torch.utils.data.TensorDataset(self.val_X, self.val_y)
        return DataLoader(dataset, batch_size=self.batch_size, shuffle=False)

    def test_dataloader(self):
        dataset = torch.utils.data.TensorDataset(self.test_X, self.test_y)
        return DataLoader(dataset, batch_size=self.batch_size, shuffle=False)

In [6]:
class DLSTMRegressor(pl.LightningModule):
    def __init__(self, input_size, hidden_size, num_layers, dropout, learning_rate):
        super().__init__()
        self.save_hyperparameters()

        # Definicja architektury DLSTM
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            dropout=dropout,
            batch_first=True
        )


        self.fc = nn.Linear(hidden_size, 1)
        self.criterion = nn.MSELoss()

    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :] # Ostatni krok czasowy
        rul_prediction = self.fc(out)
        return rul_prediction

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)

        self.log('train_loss', loss, prog_bar=True, on_step=False, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)

        loss = self.criterion(y_hat, y)
        rmse = torch.sqrt(loss)

        # Obliczenie metryki Score
        h = y_hat - y
        score1 = torch.exp(-h[h < 0] / 13) - 1
        score2 = torch.exp(h[h >= 0] / 10) - 1
        score = torch.sum(score1) + torch.sum(score2)

        self.log('val_loss', loss, prog_bar=True, on_step=False, on_epoch=True)
        self.log('val_rmse', rmse, prog_bar=True, on_step=False, on_epoch=True)
        self.log('val_score', score, prog_bar=True, on_step=False, on_epoch=True)

        return loss

    # Testowanie
    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        rmse = torch.sqrt(loss)
        h = y_hat - y
        score = torch.sum(torch.exp(-h[h < 0] / 13) - 1) + torch.sum(torch.exp(h[h >= 0] / 10) - 1)

        self.log('test_rmse', rmse, on_epoch=True)
        self.log('test_score', score, on_epoch=True)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.learning_rate)



Konfiguracja treningu

In [7]:

dm = CMAPSSDataModule(data_path='CMAPSSData', batch_size=10, sequence_length=50)

dm.setup()

drive_save_path = '/content/drive/MyDrive/01_magisterka/GSN/data_models_for_P4/logs'

model = DLSTMRegressor(
    input_size=12,
    hidden_size=100,
    num_layers=4,
    dropout=0.7,
    learning_rate=0.001
)

checkpoint_callback = ModelCheckpoint(
    dirpath=drive_save_path,
    monitor='val_rmse',
    mode='min',
    save_top_k=1,
    filename='best-dlstm-{epoch}-{val_rmse:.2f}',
)

early_stop_callback = EarlyStopping(
    monitor="val_rmse",
    min_delta=0.001,
    patience=8,
    verbose=True,
    mode="min"
)

logger = TensorBoardLogger("tb_logs", name="cmapss_lstm")

trainer = pl.Trainer(
    default_root_dir=drive_save_path,
    max_epochs=50,
    accelerator="auto",
    callbacks=[checkpoint_callback, early_stop_callback],
    gradient_clip_val=10.0,
    log_every_n_steps=1
)



INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores


Dane przygotowane!
Trening (90 silników): torch.Size([13957, 50, 12])
Walidacja (10 silników): torch.Size([1674, 50, 12])
Test (100 silników, ostatnie okna): torch.Size([93, 50, 12])


Trening

In [8]:
trainer.fit(model, dm)
print("Uruchamianie testu na najlepszym modelu...")
trainer.test(model, ckpt_path="best", datamodule=dm)

Dane przygotowane!
Trening (90 silników): torch.Size([13957, 50, 12])
Walidacja (10 silników): torch.Size([1674, 50, 12])
Test (100 silników, ostatnie okna): torch.Size([93, 50, 12])


/usr/local/lib/python3.12/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:881: Checkpoint directory /content/drive/MyDrive/01_magisterka/GSN/data_models_for_P4/logs exists and is not empty.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name      ┃ Type    ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ lstm      │ LSTM    │  288 K │ train │     0 │
│ 1 │ fc        │ Linear  │    101 │ train │     0 │
│ 2 │ criterion │ MSELoss │      0 │ train │     0 │
└───┴───────────┴─────────┴────────┴───────┴───────┘

Trainable params: 288 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 288 K                                                                                                
Total estimated model params size (MB): 1                                                                          
Modules in train mode: 3                                                                                           
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

Output()

...

INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/drive/MyDrive/01_magisterka/GSN/data_models_for_P4/logs/best-dlstm-epoch=18-val_rmse=8.29.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at /content/drive/MyDrive/01_magisterka/GSN/data_models_for_P4/logs/best-dlstm-epoch=18-val_rmse=8.29.ckpt


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_rmse         │    16.285781860351562     │
│        test_score         │    42.339576721191406     │
└───────────────────────────┴───────────────────────────┘

Dane przygotowane!
Trening (90 silników): torch.Size([13957, 50, 12])
Walidacja (10 silników): torch.Size([1674, 50, 12])
Test (100 silników, ostatnie okna): torch.Size([93, 50, 12])


[{'test_rmse': 16.285781860351562, 'test_score': 42.339576721191406}]

Pobieranie wytrenowanych modeli w celu przetestowania na zbiorze testowym NASA CMAPSS

In [13]:
! gdown --id 1XiNs-Ww0Qu0sK2u51pjadyCjep54kpLn
! gdown --id 1iu8zftxEj0Y9dFs19GGnfBQ0OxZwrOIQ


/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1XiNs-Ww0Qu0sK2u51pjadyCjep54kpLn
To: /content/ARTICLE_implementation_best-dlstm-epoch=10-val_rmse=36.96-v1.ckpt
100% 4.45M/4.45M [00:00<00:00, 13.6MB/s]
/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1iu8zftxEj0Y9dFs19GGnfBQ0OxZwrOIQ
To: /content/EXPERIMENT-best-dlstm-epoch=18-val_rmse=8.29.ckpt
100% 3.48M/3.48M [00:00<00:00, 15.9MB/s]


Interferencja na zbiorze testowym

Implementacja konfiguracji zgodnie z artykułem

In [12]:
our_implementation_based_on_article = "/content/ARTICLE_implementation_best-dlstm-epoch=10-val_rmse=36.96-v1.ckpt"

model_article = DLSTMRegressor(
    input_size=12,
    hidden_size=100,
    num_layers=5,
    dropout=0.7,
    learning_rate=0.001
)

trainer.test(model_article, ckpt_path = our_implementation_based_on_article , datamodule=dm)

INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/ARTICLE_implementation_best-dlstm-epoch=10-val_rmse=36.96-v1.ckpt
/usr/local/lib/python3.12/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:566: The dirpath has changed from '/content/drive/MyDrive/01_magisterka/GSN/data' to '/content/drive/MyDrive/01_magisterka/GSN/data_models_for_P4/logs', therefore `best_model_score`, `kth_best_model_path`, `kth_value`, `last_model_path` and `best_k_models` won't be reloaded. Only `best_model_path` will be reloaded.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at /content/ARTICLE_implementation_best-dlstm-epoch=10-val_rmse=36.96-v1.ckpt


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_rmse         │    40.726741790771484     │
│        test_score         │    1435.6119384765625     │
└───────────────────────────┴───────────────────────────┘

Dane przygotowane!
Trening (90 silników): torch.Size([13957, 50, 12])
Walidacja (10 silników): torch.Size([1674, 50, 12])
Test (100 silników, ostatnie okna): torch.Size([93, 50, 12])


[{'test_rmse': 40.726741790771484, 'test_score': 1435.6119384765625}]

Najlepszy model z naszych eskperymentów

In [15]:
our_best_experiment = "/content/EXPERIMENT-best-dlstm-epoch=18-val_rmse=8.29.ckpt"

trainer.test(model, ckpt_path = our_best_experiment , datamodule=dm)

INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/EXPERIMENT-best-dlstm-epoch=18-val_rmse=8.29.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at /content/EXPERIMENT-best-dlstm-epoch=18-val_rmse=8.29.ckpt


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_rmse         │    16.285781860351562     │
│        test_score         │    42.339576721191406     │
└───────────────────────────┴───────────────────────────┘

Dane przygotowane!
Trening (90 silników): torch.Size([13957, 50, 12])
Walidacja (10 silników): torch.Size([1674, 50, 12])
Test (100 silników, ostatnie okna): torch.Size([93, 50, 12])


[{'test_rmse': 16.285781860351562, 'test_score': 42.339576721191406}]

In [ ]:
# %load_ext tensorboard

# %tensorboard --logdir lightning_logs
